In [25]:
import random
import crcmod

def reverse_bits(value, bits):
    reversed_bits = 0
    for i in range(bits):
        if value & (1 << i):
            reversed_bits |= (1 << (bits - 1 - i))
    return reversed_bits

def create_crc_table(width, poly, refin=False):
    global crc_table
    crc_table = []  # 清空表
    
    mask = (1 << width) - 1
    poly = poly & mask
    # Reverse the polynomial once if refin is True
    if refin:
        poly = reverse_bits(poly, width)
        
    for byte in range(256):
        if refin:
            # Reflect input byte and process
            remain = reverse_bits(byte, 8)
            for _ in range(8):
                if remain & 1:
                    remain = (remain >> 1) ^ poly
                else:
                    remain >>= 1
        else:
            # MSB-first processing
            remain = byte << (width - 8)
            for _ in range(8):
                if remain & (1 << (width - 1)):
                    remain = (remain << 1) ^ poly
                else:
                    remain <<= 1
                remain &= mask
        crc_table.append(remain & mask)

def crc_calculate(data_bytes, width, init, xor_out, refin, refout):
    mask = (1 << width) - 1
    crc = init & mask

    for byte in data_bytes:
        if refin:
            # 反转输入，使用LSB优先处理
            byte_to_use = reverse_bits(byte, 8)
            index = (crc & 0xFF) ^ byte_to_use
            crc = (crc >> 8) ^ crc_table[index]
        else:
            # MSB优先处理
            index = ((crc >> (width - 8)) ^ byte) & 0xFF
            crc = ((crc << 8) & mask) ^ crc_table[index]

    # 修正refout处理：只在refout与refin不同时才进行反转
    if refout != refin:
        crc = reverse_bits(crc, width)
    
    crc ^= xor_out  
    crc &= mask
    return crc

def run_monitor_tests(num_tests=100, max_length=100):
    """
    :param num_tests: 随机测试的次数
    :param max_length: 随机生成数据的最大长度（字节数）
    """
    # 根据参数创建查找表
    create_crc_table(16, 0x1021, refin=False)

    # 利用 crcmod 生成一个参考的 CRC-CCITT 计算函数
    crc16_func = crcmod.mkCrcFun(0x11021, initCrc=0xFFFF, rev=False, xorOut=0x0000)
    
    # 进行多次随机测试
    for i in range(num_tests):
        length = random.randint(1, max_length)
        # 随机生成一个 0~255 的字节列表
        data = [random.randint(0, 255) for _ in range(length)]
        # 计算我们实现的CRC值
        my_crc = crc_calculate(data, width=16, init=0xFFFF, xor_out=0x0000, refin=False, refout=False)
        # 计算参考库的CRC值
        expected_crc = crc16_func(bytes(data))
        if my_crc != expected_crc:
            print(f"Test {i+1} FAILED:")
            print(f"  Data:        {data}")
            print(f"  My CRC:      0x{my_crc:04X}")
            print(f"  Expected CRC:0x{expected_crc:04X}")
        else:
            print(f"Test {i+1} passed.")
# 测试正常计算
create_crc_table(16, 0x1021, refin=False)
data_bytes = [0x33]
calculated_crc = crc_calculate(data_bytes, 16, 0xFFFF, 0x0000, False, False)
print(f"Standard mode: {calculated_crc:04x}")

# 测试反射输入
create_crc_table(16, 0x1021, refin=True)
calculated_crc = crc_calculate(data_bytes, 16, 0xFFFF, 0x0000, True, True)
print(f"Reflected mode (refin=True, refout=True): {calculated_crc:04x}")

# 测试混合模式
create_crc_table(16, 0x1021, refin=True)
calculated_crc = crc_calculate(data_bytes, 16, 0xFFFF, 0x0000, True, False)
print(f"Mixed mode (refin=True, refout=False): {calculated_crc:04x}")

# 使用标准库比对
crc16_func_standard = crcmod.mkCrcFun(0x11021, initCrc=0xFFFF, rev=False, xorOut=0x0000)
crc16_func_reflected = crcmod.mkCrcFun(0x11021, initCrc=0xFFFF, rev=True, xorOut=0x0000)
print(f"Expected standard: {crc16_func_standard(bytes(data_bytes)):04x}")
print(f"Expected reflected: {crc16_func_reflected(bytes(data_bytes)):04x}")

Standard mode: e7c0
Reflected mode (refin=True, refout=True): 0c9f
Mixed mode (refin=True, refout=False): f930
Expected standard: e7c0
Expected reflected: 0c9f


In [1]:
import crcmod

def reverse_bits(x, num_bits):
    """反转指定位数的位序"""
    reversed_x = 0
    for i in range(num_bits):
        reversed_x |= ((x >> i) & 1) << (num_bits - 1 - i)
    return reversed_x


def crc_process_byte(crc, byte, poly, width, refin):
    """处理单个字节的CRC计算"""
    # 1. 如果需要反转输入
    if refin:
        byte = reverse_bits(byte, 8)
    
    # 2. 直接将字节与CRC高位进行异或（避免一位一位处理）
    crc ^= (byte << (width - 8))
    
    # 3. 处理8个位
    for _ in range(8):
        # 判断最高位，使用位移判断避免额外计算
        if crc & (1 << (width - 1)):
            # 左移+异或多项式
            crc = ((crc << 1) ^ poly) & ((1 << width) - 1)
        else:
            # 仅左移
            crc = (crc << 1) & ((1 << width) - 1)
    
    return crc

def calculate_crc(data_bytes, width, poly, init, refin, refout, xorout):
    """计算字节序列的CRC校验值"""
    # 确保poly不包含最高位(如果已经包含)
    poly = poly & ((1 << width) - 1)
    
    crc = init
    for byte in data_bytes:
        crc = crc_process_byte(crc, byte, poly, width, refin)
    if refout:
        crc = reverse_bits(crc, width)
    crc ^= xorout
    crc &= (1 << width) - 1  # 确保结果在低width位
    return crc

# 示例用法
if __name__ == "__main__":
    # 示例参数（以CRC-8为例）
    width = 16
    poly = 0x10c21  # 多项式 x^8 + x^2 + x + 1 (隐式最高位)
    init = 0xffff
    xorout = 0x0000

    # 输入数据（假设输入S018F0转换为字节数组）
    input_data = [0x71,0xFA,0x96,0x59,0x91,0x93,0xB2,0xD1,0x35]
    crc_result_standard = calculate_crc(input_data, width, poly, init, False, False, xorout)
    print(f"CRC结果: 0x{crc_result_standard:02X}")
    crc_result_reflected = calculate_crc(input_data, width, poly, init, True, True, xorout)
    print(f"CRC结果: 0x{crc_result_reflected:02X}")
    crc_result_mixed1=calculate_crc(input_data,width,poly,init,True,False,xorout)
    print(f"CRC结果: 0x{crc_result_mixed1:02X}")
    crc_result_mixed2=calculate_crc(input_data,width,poly,init,False,True,xorout)
    print(f"CRC结果: 0x{crc_result_mixed2:02X}")
    
    crc16_func_standard = crcmod.mkCrcFun(poly, initCrc=0xffff, rev=False, xorOut=0x0000)
    crc16_func_reflected = crcmod.mkCrcFun(poly, initCrc=0xffff, rev=True, xorOut=0x0000)
    print(f"Expected standard: {crc16_func_standard(bytes(input_data)):04x}")
    print(f"Expected reflected: {crc16_func_reflected(bytes(input_data)):04x}")

CRC结果: 0xBB97
CRC结果: 0x53C5
CRC结果: 0xA3CA
CRC结果: 0xE9DD
Expected standard: bb97
Expected reflected: 53c5
